# 雪球期权
![grid](../note_pic/snowball.png)

## 到期收益分析
| 情景 | 具体分析| 合约实际结束时间   |       投资者收益       |  是否获得正收益  |
|:--:|:---:|:---:|:-----------------:|:---------:|
| 敲出 |从第三个月开始，10个敲出观察日中任何一天标的资产大于或等于沪深300初始价格的103%|T日合约提前终止| 25% * T/365 * 本金  |     是     |
| 未敲入且未敲出 |标的价格每日都不低于初始价格的80%，且每个敲出观察日都小于初始价格的103%|合约持有到期|25% * 360/365 * 本金|是(收益最高)|
|敲入且未敲出|任一交易日跌破初始价格的80%，且10个敲出观察日都小于初始价格的103%，<mark>标的资产的期末价格小于期初价格<mark>|合约持有到期|(期末价格/期初价格 - 1) * 本金|否(发生亏损)|
|敲入且未敲出|任一交易日跌破初始价格的80%，且10个敲出观察日都小于初始价格的103%，<mark>标的资产的期末价格处于期初价格和敲出价格之间</mark>|合约持有到期|0|否(保本)|  

1. 触发敲出，产品提前终止
根据图中案例产品，从第三个月开始，***每月***观察敲入 + ***每日***观察敲出。当标的资产价格大于敲出价格，投资者将会获得25%的年化收益率，产品提前赎回。  
产品在存续期间可能发生两种情况：在触发敲出前***从未发生敲入***；或者是***曾经发生过敲入***。在这两种情况下，***收益均为25% * T/365 * 本金，并全额收回本金***   
![](../note_pic/%E6%88%AA%E5%B1%8F2022-08-16%2016.06.10.png)

2. 从未发生敲入或敲出
存续期间从未发生敲入或敲出，直至产品到期。收益为***收益均为25% * 360/365 * 本金，并全额收回本金***  
![](../note_pic/%E6%88%AA%E5%B1%8F2022-08-16%2016.09.52.png)

3. 敲入且未敲出
此时，投资者的收益由标的资产的到期价格决定。若标的资产的到期价格高于期初价格但是低于敲出价格，则收回本金且没有任何额外收益。  
![](../note_pic/%E6%88%AA%E5%B1%8F2022-08-16%2016.14.24.png)

若标的资产的到期价格低于起初价格，此时不仅不能获得收益，还需要承担资产在存续期间的跌幅所对应的名义本金价值，亏损金额为(期末价格/期初价格 - 1) * 名义本金，这也是雪球产品本金发生亏损的唯一情况  
![](../note_pic/%E6%88%AA%E5%B1%8F2022-08-16%2016.17.02.png)

In [ ]:
import numpy as np

In [ ]:
# MC
